In [16]:
import os
import tensorflow as tf
import numpy as np
from rnn_model.attention_lstm import bilstm_with_attention
from tensorflow.contrib import rnn
import pickle
import konlpy
from konlpy.tag import Twitter

In [17]:
pos_tagger = Twitter()

C:\Users\user\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
with open('dict','rb') as fp:
    word_dict = pickle.load(fp)

In [18]:
def tokenize(doc):

    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]
        

In [38]:
a = '일단 너한테 많이 미안해 내가 이기적이라서 갤 잊을 수 있게 도와줘. 원하지 않게 맞이했던 이별이라서 날 안 놓아줘'

In [39]:
input_sentence = [tokenize(a)]
tokens = list(map(lambda d: list(map(lambda w: word_dict.get(w, word_dict["<unk>"]), d)), input_sentence))
tokens = list(map(lambda d: d + [word_dict["<eos>"]], tokens))
tokens = list(map(lambda d: d + (80 - len(d)) * [word_dict["<pad>"]], tokens))

In [40]:
tokens

[[494,
  151,
  181,
  69,
  1,
  46,
  6,
  103,
  2572,
  1531,
  864,
  708,
  64,
  9,
  1833,
  1,
  904,
  66,
  3844,
  3,
  4672,
  1531,
  144,
  18,
  356,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [41]:
num_class=30
embedding_size = 256
BATCH_SIZE = 64
NUM_EPOCHS = 1
WORD_MAX_LEN = 80
vocabulary_size = len(word_dict)
from tensorflow.contrib import rnn
       
      
num_class=30
embedding_size = 256
num_hidden = 256
num_layers = 2
learning_rate = 1e-3
tf.reset_default_graph()
x = tf.placeholder(tf.int32, [None, 80], name="x")
x_len = tf.reduce_sum(tf.sign(x), 1)
y = tf.placeholder(tf.int32, [None], name="y")
is_training = tf.placeholder(tf.bool, [], name="is_training")
global_step = tf.Variable(0, trainable=False)
keep_prob = tf.where(is_training, 0.5, 1.0)

with tf.name_scope("embedding"):
    init_embeddings = tf.random_uniform([vocabulary_size, embedding_size])
    embeddings = tf.get_variable("embeddings", initializer=init_embeddings)
    x_emb = tf.nn.embedding_lookup(embeddings, x)

with tf.name_scope("birnn"):
    fw_cells = [rnn.BasicLSTMCell(num_hidden) for _ in range(num_layers)]
    bw_cells = [rnn.BasicLSTMCell(num_hidden) for _ in range(num_layers)]
    fw_cells = [rnn.DropoutWrapper(cell, output_keep_prob=keep_prob) for cell in fw_cells]
    bw_cells = [rnn.DropoutWrapper(cell, output_keep_prob=keep_prob) for cell in bw_cells]

rnn_outputs, _, _ = rnn.stack_bidirectional_dynamic_rnn(
fw_cells, bw_cells, x_emb, sequence_length=x_len, dtype=tf.float32)

with tf.name_scope("attention"):
    attention_score = tf.nn.softmax(tf.layers.dense(rnn_outputs, 1, activation=tf.nn.tanh), axis=1)
    attention_out = tf.squeeze(
    tf.matmul(tf.transpose(rnn_outputs, perm=[0, 2, 1]), attention_score),axis=-1)


with tf.name_scope("output"):
    logits = tf.layers.dense(attention_out, num_class, activation=tf.nn.softmax)

saver = tf.train.Saver()
init = tf.global_variables_initializer()
modelName = "BILSTM_attention/BILSTM_attention.ckpt-35125"
sess = tf.Session()
sess.run(init)
saver.restore(sess, modelName)


INFO:tensorflow:Restoring parameters from BILSTM_attention/BILSTM_attention.ckpt-35125


In [42]:
probabilities = sess.run(tf.nn.top_k(logits,5,sorted=True), feed_dict = {x: np.array(tokens).reshape(1,80),is_training:False})

attention_weights = sess.run(attention_score,feed_dict = {x: np.array(tokens).reshape(1,80),is_training:False})

In [43]:
probabilities.indices

array([[ 9, 13, 24, 19, 10]])

In [44]:
a = []
for i in range(5):
    if probabilities.indices[0][i] == 1:
        a.append(u'\U0001F44D')
    elif probabilities.indices[0][i] == 13:
        a.append(u'\U0001F622')
    elif probabilities.indices[0][i] == 0:
        a.append(u'\U0001F44C')
    elif probabilities.indices[0][i] == 2:
        a.append(u'\U0001F601')
    elif probabilities.indices[0][i] == 3:
        a.append(u'\U0001F602')
    elif probabilities.indices[0][i] == 4:
        a.append(u'\U0001F608')
    elif probabilities.indices[0][i] == 5:
        a.append(u'\U0001F60A')
    elif probabilities.indices[0][i] == 6:
        a.append(u'\U0001F60D')
    elif probabilities.indices[0][i] == 7:
        a.append(u'\U0001F60F')
    elif probabilities.indices[0][i] == 8:
        a.append(u'\U0001F612')
    elif probabilities.indices[0][i] == 9:
        a.append(u'\U0001F614')
    elif probabilities.indices[0][i] == 10:
        a.append(u'\U0001F615')
    elif probabilities.indices[0][i] == 11:
        a.append(u'\U0001F618')
    elif probabilities.indices[0][i] == 12:
        a.append(u'\U0001F621')
    elif probabilities.indices[0][i] == 14:
        a.append(u'\U0001F629')
    elif probabilities.indices[0][i] == 15:
        a.append(u'\U0001F62D')
    elif probabilities.indices[0][i] == 16:
        a.append(u'\U0001F633')
    elif probabilities.indices[0][i] == 17:
        a.append(u'\U0001F635')
    elif probabilities.indices[0][i] == 18:
        a.append(u'\U0001F637')
    elif probabilities.indices[0][i] == 19:
        a.append(u'\U0001F64F')
    elif probabilities.indices[0][i] == 20:
        a.append(u'\U0001F605')
    elif probabilities.indices[0][i] == 21:
        a.append(u'\U0001F60B')
    elif probabilities.indices[0][i] == 22:
        a.append(u'\U0001F60E')
    elif probabilities.indices[0][i] == 23:
        a.append(u'\U0001F61C')
    elif probabilities.indices[0][i] == 24:
        a.append(u'\U0001F61E')
    elif probabilities.indices[0][i] == 25:
        a.append(u'\U0001F603')
    elif probabilities.indices[0][i] == 26:
        a.append(u'\U0001F648')
    elif probabilities.indices[0][i] == 27:
        a.append(u'\U0001F64C')
    elif probabilities.indices[0][i] == 28:
        a.append(u'\U0001F914')
    elif probabilities.indices[0][i] == 29:
        a.append(u'\U0001F611')
        
print(a)


['😔', '😢', '😞', '🙏', '😕']


In [51]:
attention = [a[0] for a in attention_weights[0]]
enumerated_attention = list(enumerate(attention))
index_sorted_attention= sorted(enumerated_attention, key=lambda x:x[1],reverse=True)
top3_index = [x[0] for x in index_sorted_attention[:5]]

In [52]:
for i in range(5):
    print(input_sentence[0][top3_index[i]])

안/Noun
도와주다/Verb
수/Noun
원하다/Adjective
이별/Noun
